In [1]:
import requests
import os
import shutil
import zipfile
import geopandas as gpd
import pandas as pd
import time
import numpy as np
import io
import re

## Step One: Data Preparation / Web Scraping

Below the code below will: 1) download all US state shapefiles for mapping local districts, 2) create a GeoPandas dataframe which contains all of the map data.

In [40]:
fips_codes = pd.read_csv('State_FIPS_codes.csv')
fips_codes['FIPS State Numeric Code'] = fips_codes['FIPS State Numeric Code'].astype(str)
fips_codes.dtypes

Name                       object
FIPS State Numeric Code    object
Official USPS Code         object
dtype: object

In [2]:
# Download shapefiles
base_url = 'http://www2.census.gov/geo/tiger/TIGER_RD18/LAYER/SLDL/'
for i in range(1,100):
    local_path = 'tmp/'
    padded = str(i).rjust(2, '0')
    layer = 'tl_rd22_'+padded+'_sldl.zip'
    url = base_url+layer

    if os.path.isfile('tmp/'+layer):
        #print(layer)
        continue
    r = requests.get(url)
    if r.status_code == 200:
        z = zipfile.ZipFile(io.BytesIO(r.content))
        z.extractall(path=local_path)
    else:
        continue

3
7
11
14
31
43
52
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99


In [3]:
## Could be optimized I'm sure, but this still runs quickly to concatenate
## the shapefiles togther.

for i in range(1,100):
    padded = str(i).rjust(2, '0')
    layer = 'tl_rd22_'+padded+'_sldl.shp'
    if os.path.isfile('tmp/'+layer):
        print("getting file... ", str(i))
        if i == 1:
            districts = gpd.read_file("tmp/"+layer)
        else:
            new_districts = gpd.read_file("tmp/"+layer)
            #districts = districts.concat(new_districts)
            districts = gpd.GeoDataFrame( pd.concat( [districts, new_districts], ignore_index=True) )        

getting file...  1
getting file...  2
getting file...  4
getting file...  5
getting file...  6
getting file...  8
getting file...  9
getting file...  10
getting file...  12
getting file...  13
getting file...  15
getting file...  16
getting file...  17
getting file...  18
getting file...  19
getting file...  20
getting file...  21
getting file...  22
getting file...  23
getting file...  24
getting file...  25
getting file...  26
getting file...  27
getting file...  28
getting file...  29
getting file...  30
getting file...  32
getting file...  33
getting file...  34
getting file...  35
getting file...  36
getting file...  37
getting file...  38
getting file...  39
getting file...  40
getting file...  41
getting file...  42
getting file...  44
getting file...  45
getting file...  46
getting file...  47
getting file...  48
getting file...  49
getting file...  50
getting file...  51
getting file...  53
getting file...  54
getting file...  55
getting file...  56
getting file...  72


In [4]:
## Extract the number from the district to match later
districts['District Number'] = districts.NAMELSAD.str.extract('(\d+)')

In [5]:
## Take a look at the data
districts.head()

,STATEFP,SLDLST,GEOID,NAMELSAD,LSAD,LSY,MTFCC,FUNCSTAT,ALAND,AWATER,INTPTLAT,INTPTLON,geometry,District Number
0,01,003,01003,State House District 3,LL,2022,G5220,N,352124546,28583926,+34.7764307,-087.7624918,"POLYGON ((-87.93346 34.83890, -87.93302 34.839...",3
1,01,005,01005,State House District 5,LL,2022,G5220,N,499374971,30532838,+34.8159775,-086.9344872,"POLYGON ((-87.06353 34.91379, -87.06344 34.914...",5
2,01,006,01006,State House District 6,LL,2022,G5220,N,376354421,1695809,+34.9011048,-086.7613086,"POLYGON ((-86.87772 34.99221, -86.87717 34.992...",6
3,01,007,01007,State House District 7,LL,2022,G5220,N,2158056827,106725966,+34.5506815,-087.3337337,"POLYGON ((-87.64340 34.79285, -87.64263 34.793...",7
4,01,008,01008,State House District 8,LL,2022,G5220,N,244912636,2249707,+34.4461776,-087.0769893,"POLYGON ((-87.11016 34.37777, -87.11013 34.379...",8


## Part Two: OpenStates API Data Pulls
This section of the notebook will pull: 1) all states available and their jurisdictionID (as represented in OpenStates), 2) Create a DataFrame which is ready to be joined to business data

In [7]:
state_r = requests.get('https://v3.openstates.org/jurisdictions?classification=state&page=1&per_page=52&apikey=0d020ec0-648a-4e93-9695-7192883991d8')
state_r_json = state_r.json()

index = 0
juris_dict = {}

for i in state_r_json['results']:
        if index == 1:
            print(i)
        ind = index
        name = i['name']
        juris = i['id']
        juris_dict[name] = juris


In [8]:
juris_dict

{'Alabama': 'ocd-jurisdiction/country:us/state:al/government',
 'Alaska': 'ocd-jurisdiction/country:us/state:ak/government',
 'Arizona': 'ocd-jurisdiction/country:us/state:az/government',
 'Arkansas': 'ocd-jurisdiction/country:us/state:ar/government',
 'California': 'ocd-jurisdiction/country:us/state:ca/government',
 'Colorado': 'ocd-jurisdiction/country:us/state:co/government',
 'Connecticut': 'ocd-jurisdiction/country:us/state:ct/government',
 'Delaware': 'ocd-jurisdiction/country:us/state:de/government',
 'District of Columbia': 'ocd-jurisdiction/country:us/district:dc/government',
 'Florida': 'ocd-jurisdiction/country:us/state:fl/government',
 'Georgia': 'ocd-jurisdiction/country:us/state:ga/government',
 'Hawaii': 'ocd-jurisdiction/country:us/state:hi/government',
 'Idaho': 'ocd-jurisdiction/country:us/state:id/government',
 'Illinois': 'ocd-jurisdiction/country:us/state:il/government',
 'Indiana': 'ocd-jurisdiction/country:us/state:in/government',
 'Iowa': 'ocd-jurisdiction/count

In [50]:
## Pull data on every legislator, including their role (State Senate or Representative), party and jurisdiction

names = {}
ids = {}
counties = {}

index = 0
index_c = 0

for state in juris_dict:
    print(state)
    jur = juris_dict[state]
    string = 'https://v3.openstates.org/people?jurisdiction='+jur+'&page=1&per_page=50&org_classification=lower&apikey=0d020ec0-648a-4e93-9695-7192883991d8'
    try:
        init_r = requests.get(string)
        init_r_json = init_r.json()
        pages = init_r_json['pagination']['max_page']
    except:
        print(string)
        continue

    # Save names, the ids (for future API pulls), counties for joining later
    for i in init_r_json['results']:
        ind = index
        id_ = i['id']
        name = i['name']
        role = i['current_role']['title']
        district = i['current_role']['district']
        party = i['party']
        juris = i['jurisdiction']['id']
        names[ind] = {'name':name, 'role': role, 'party': party, 'district':district}
        ids[name] = {'id': id_, 'jurisdiction': juris}
        index += 1
        ## Sleep for 1 second to avoid API calls being throttled
        time.sleep(1)

    # API call will have more than one page at times, if it does, capture the additional pages of information
    if pages == 1:
        print('nothing to do')
    else:
        for i in range(2,pages+1):
            new_r = requests.get('https://v3.openstates.org/people?jurisdiction='+jur+'&page='+str(i)+'&per_page=50&org_classification=lower&apikey=0d020ec0-648a-4e93-9695-7192883991d8')
            new_r_json = new_r.json()
            for i in new_r_json['results']:
                ind = index
                id_ = i['id']
                name = i['name']
                role = i['current_role']['title']
                district = i['current_role']['district']
                party = i['party']
                names[ind] = {'name':name,'role': role, 'party':party, 'district':district}
                juris = i['jurisdiction']['id']
                ids[name] = {'id': id_, 'jurisdiction': juris}
                index += 1
                time.sleep(1)
    

Alabama
Alaska
nothing to do
Arizona
Arkansas
California
Colorado
Connecticut
Delaware
nothing to do
District of Columbia
nothing to do
Florida
Georgia
Hawaii
Idaho
Illinois
Indiana
Iowa
Kansas
Kentucky
Louisiana
Maine
Maryland
Massachusetts
Michigan
Minnesota
Mississippi
Missouri
Montana
Nebraska
nothing to do
Nevada
nothing to do
New Hampshire
New Jersey
New Mexico
New York
North Carolina
North Dakota
Ohio
Oklahoma
Oregon
Pennsylvania
Puerto Rico
Rhode Island
South Carolina
South Dakota
Tennessee
Texas
Utah
Vermont
Virginia
Washington
West Virginia
Wisconsin
Wyoming


In [51]:
# Show Dataframe for representatives
rep_df = pd.DataFrame.from_dict(names, orient='index')
rep_df

,name,role,party,district
0,Adline Clarke,Representative,Democratic,97
1,Alan Baker,Representative,Republican,66
2,Allen Treadaway,Representative,Republican,51
3,Andy Whitt,Representative,Republican,6
4,Anthony Daniels,Representative,Democratic,53
...,...,...,...,...
5452,Tom Walters,Representative,Republican,38
5453,Tomi Strock,Representative,Republican,6
5454,Tony Locke,Representative,Republican,35
5455,Tony Niemiec,Representative,Republican,60


In [52]:
# Show Other Dictionary of data
id_df = pd.DataFrame.from_dict(ids, orient='index')
id_df

,id,jurisdiction
Adline Clarke,ocd-person/4cb1aeb5-d822-4f2c-a7e6-ae2a132acf17,ocd-jurisdiction/country:us/state:al/government
Alan Baker,ocd-person/cb2488d5-f0b6-4e9f-948b-ff7e9f4e1907,ocd-jurisdiction/country:us/state:al/government
Allen Treadaway,ocd-person/3f96a690-e9a3-4774-8484-49a5a079b18c,ocd-jurisdiction/country:us/state:al/government
Andy Whitt,ocd-person/c128ed12-dd1b-4b5f-8c64-9d86bf9d6580,ocd-jurisdiction/country:us/state:al/government
Anthony Daniels,ocd-person/95f7541d-5429-41aa-85e5-7fc9cf33d688,ocd-jurisdiction/country:us/state:al/government
...,...,...
Tom Walters,ocd-person/769a4c49-d52b-467e-a801-ee3c95a2dd8c,ocd-jurisdiction/country:us/state:wy/government
Tomi Strock,ocd-person/d1ac363d-41ed-4c2c-a661-6e400d0b7d57,ocd-jurisdiction/country:us/state:wy/government
Tony Locke,ocd-person/0cd9d0d3-09eb-4cf1-8e5b-bfc15a71b337,ocd-jurisdiction/country:us/state:wy/government
Tony Niemiec,ocd-person/7be1f0fc-a31f-45b8-b686-fc580d3cb8bb,ocd-jurisdiction/country:us/state:wy/government


In [53]:
# Create column that contains the state 
new_column = []    
for value in id_df['jurisdiction']:
    try:
        new_column.append(str.upper(re.search(':([a-z]+)/government', value).group(1)))
    except:
        print(value)

id_df['state'] = new_column
id_df = id_df.reset_index()

In [54]:
# Combine data together to a dataframe
rep_df_2 = rep_df.merge(id_df, left_on='name', right_on = 'index')
rep_df_fips = rep_df_2.merge(fips_codes, left_on='state',right_on='Official USPS Code')
rep_df_fips['fips_str'] = rep_df_fips['FIPS State Numeric Code'].str.zfill(2)
rep_df_fips

,name,role,party,district,index,id,jurisdiction,state,Name,FIPS State Numeric Code,Official USPS Code,fips_str
0,Adline Clarke,Representative,Democratic,97,Adline Clarke,ocd-person/4cb1aeb5-d822-4f2c-a7e6-ae2a132acf17,ocd-jurisdiction/country:us/state:al/government,AL,Alabama,1,AL,01
1,Alan Baker,Representative,Republican,66,Alan Baker,ocd-person/cb2488d5-f0b6-4e9f-948b-ff7e9f4e1907,ocd-jurisdiction/country:us/state:al/government,AL,Alabama,1,AL,01
2,Allen Treadaway,Representative,Republican,51,Allen Treadaway,ocd-person/3f96a690-e9a3-4774-8484-49a5a079b18c,ocd-jurisdiction/country:us/state:al/government,AL,Alabama,1,AL,01
3,Andy Whitt,Representative,Republican,6,Andy Whitt,ocd-person/c128ed12-dd1b-4b5f-8c64-9d86bf9d6580,ocd-jurisdiction/country:us/state:al/government,AL,Alabama,1,AL,01
4,Anthony Daniels,Representative,Democratic,53,Anthony Daniels,ocd-person/95f7541d-5429-41aa-85e5-7fc9cf33d688,ocd-jurisdiction/country:us/state:al/government,AL,Alabama,1,AL,01
...,...,...,...,...,...,...,...,...,...,...,...,...
5401,Tom Walters,Representative,Republican,38,Tom Walters,ocd-person/769a4c49-d52b-467e-a801-ee3c95a2dd8c,ocd-jurisdiction/country:us/state:wy/government,WY,Wyoming,56,WY,56
5402,Tomi Strock,Representative,Republican,6,Tomi Strock,ocd-person/d1ac363d-41ed-4c2c-a661-6e400d0b7d57,ocd-jurisdiction/country:us/state:wy/government,WY,Wyoming,56,WY,56
5403,Tony Locke,Representative,Republican,35,Tony Locke,ocd-person/0cd9d0d3-09eb-4cf1-8e5b-bfc15a71b337,ocd-jurisdiction/country:us/state:wy/government,WY,Wyoming,56,WY,56
5404,Tony Niemiec,Representative,Republican,60,Tony Niemiec,ocd-person/7be1f0fc-a31f-45b8-b686-fc580d3cb8bb,ocd-jurisdiction/country:us/state:wy/government,WY,Wyoming,56,WY,56


## Step 3: Combine Data Together
Combine data from step 1 & 2 to get a shapefile by district and representative

In [55]:
district_full = rep_df_fips.merge(districts, left_on=['district','fips_str'], right_on=['District Number', 'STATEFP'])
district_full.columns

Index(['name', 'role', 'party', 'district', 'index', 'id', 'jurisdiction',
       'state', 'Name', 'FIPS State Numeric Code', 'Official USPS Code',
       'fips_str', 'STATEFP', 'SLDLST', 'GEOID', 'NAMELSAD', 'LSAD', 'LSY',
       'MTFCC', 'FUNCSTAT', 'ALAND', 'AWATER', 'INTPTLAT', 'INTPTLON',
       'geometry', 'District Number'],
      dtype='object')

In [56]:
district_full[['name','role','party','district','state','STATEFP','SLDLST','GEOID','geometry']]

,name,role,party,district,state,STATEFP,SLDLST,GEOID,geometry
0,Adline Clarke,Representative,Democratic,97,AL,01,097,01097,"POLYGON ((-88.11462 30.68308, -88.11325 30.683..."
1,Alan Baker,Representative,Republican,66,AL,01,066,01066,"POLYGON ((-87.73131 30.90321, -87.73095 30.903..."
2,Allen Treadaway,Representative,Republican,51,AL,01,051,01051,"POLYGON ((-86.96357 33.72058, -86.96326 33.721..."
3,Andy Whitt,Representative,Republican,6,AL,01,006,01006,"POLYGON ((-86.87772 34.99221, -86.87717 34.992..."
4,Anthony Daniels,Representative,Democratic,53,AL,01,053,01053,"POLYGON ((-86.67282 34.76187, -86.67280 34.763..."
...,...,...,...,...,...,...,...,...,...
4336,Tom Walters,Representative,Republican,38,WY,56,038,56038,"POLYGON ((-107.54353 42.74998, -107.54353 42.7..."
4337,Tomi Strock,Representative,Republican,6,WY,56,006,56006,"POLYGON ((-105.77265 42.72103, -105.76927 42.7..."
4338,Tony Locke,Representative,Republican,35,WY,56,035,56035,"POLYGON ((-106.33456 42.78819, -106.33451 42.7..."
4339,Tony Niemiec,Representative,Republican,60,WY,56,060,56060,"POLYGON ((-109.50244 41.47799, -109.50244 41.4..."


## Step 4: Gather Business Sector Data and Combine to County Level
Data for business sectors is at the county level and not district, we will use a mapping file provided by the Census to map these together, and be ready to combine into our final ShapeFile.

In [66]:
url = 'https://www2.census.gov/geo/docs/maps-data/data/rel2020/cd-sld/tab20_sldl202220_county20_natl.txt'
#req = requests.get('https://www2.census.gov/geo/docs/maps-data/data/rel2020/cd-sld/tab20_sldl202220_county20_natl.txt'


In [83]:
## Read CSV from URL, combine with Rep data to connect representative to county
county_to_geo = pd.read_csv(url,delimiter='|', header=0)
rep_with_county = district_full.merge(county_to_geo,left_on='GEOID',right_on='GEOID_SLDL2022_20')
rep_with_county['county_short'] = rep_with_county['NAMELSAD_COUNTY_20'].str.replace(r' County$','')

/var/folders/n2/21777nbs0wx_k3zppskxw6540000gn/T/ipykernel_69851/3693185094.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  rep_with_county['county_short'] = rep_with_county['NAMELSAD_COUNTY_20'].str.replace(r' County$','')


In [113]:
rep_with_county.head()

,name,role,party,district,index,id,jurisdiction,state,Name,FIPS State Numeric Code,...,GEOID_COUNTY_20,NAMELSAD_COUNTY_20,AREALAND_COUNTY_20,AREAWATER_COUNTY_20,MTFCC_COUNTY_20,CLASSFP_COUNTY_20,FUNCSTAT_COUNTY_20,AREALAND_PART,AREAWATER_PART,county_short
0,Adline Clarke,Representative,Democratic,97,Adline Clarke,ocd-person/4cb1aeb5-d822-4f2c-a7e6-ae2a132acf17,ocd-jurisdiction/country:us/state:al/government,AL,Alabama,1,...,1097,Mobile County,3184064139,1074011171,G4020,H1,A,131313661,42560622,Mobile
1,Alan Baker,Representative,Republican,66,Alan Baker,ocd-person/cb2488d5-f0b6-4e9f-948b-ff7e9f4e1907,ocd-jurisdiction/country:us/state:al/government,AL,Alabama,1,...,1003,Baldwin County,4117656199,1132956041,G4020,H1,A,1100580784,36591076,Baldwin
2,Alan Baker,Representative,Republican,66,Alan Baker,ocd-person/cb2488d5-f0b6-4e9f-948b-ff7e9f4e1907,ocd-jurisdiction/country:us/state:al/government,AL,Alabama,1,...,1053,Escambia County,2448595161,20024887,G4020,H1,A,1663563050,11665994,Escambia
3,Allen Treadaway,Representative,Republican,51,Allen Treadaway,ocd-person/3f96a690-e9a3-4774-8484-49a5a079b18c,ocd-jurisdiction/country:us/state:al/government,AL,Alabama,1,...,1073,Jefferson County,2878852142,32681784,G4020,H1,A,515594222,1795940,Jefferson
4,Andy Whitt,Representative,Republican,6,Andy Whitt,ocd-person/c128ed12-dd1b-4b5f-8c64-9d86bf9d6580,ocd-jurisdiction/country:us/state:al/government,AL,Alabama,1,...,1083,Limestone County,1450296914,122310594,G4020,H1,A,169197056,851076,Limestone


In [87]:
## Data file which contains Business Sector information
## From https://www2.census.gov/programs-surveys/cbp/datasets/2020/cbp20co.zip
cbp = pd.read_csv('/Users/katiefinnegan/Desktop/OMSA/Capstone/cbp20co.txt')

In [108]:
# Convert to correct datatypes
cbp['naics_num'] = cbp.naics.str.extract('(\d+)').fillna(0).astype(int)
cbp['fipstate'] = cbp.fipstate.astype(str)
cbp['fipscty'] = cbp.fipscty.astype(str)

In [109]:
cbp.head()

,fipstate,fipscty,naics,emp_nf,emp,qp1_nf,qp1,ap_nf,ap,est,...,n50_99,n100_249,n250_499,n500_999,n1000,n1000_1,n1000_2,n1000_3,n1000_4,naics_num
0,1,1,------,G,11265,G,94865,G,385785,879,...,32,9,N,N,N,N,N,N,N,0
1,1,1,11----,H,92,G,1183,H,5232,10,...,N,N,N,N,N,N,N,N,N,11
2,1,1,113///,H,82,G,1075,G,4741,7,...,N,N,N,N,N,N,N,N,N,113
3,1,1,1133//,H,82,G,1075,G,4741,7,...,N,N,N,N,N,N,N,N,N,1133
4,1,1,11331/,H,82,G,1075,G,4741,7,...,N,N,N,N,N,N,N,N,N,11331


In [111]:
## Pull the top level business secor information (no sub-sections, although this could be expanded to include that)
## Create a GEOID column to join to the dataframe from step 3
high_lev = cbp[cbp['naics_num'] < 100]
high_lev['state'] = high_lev.fipstate.str.rjust(2, '0')
high_lev['county'] = high_lev.fipscty.str.rjust(3,'0')
high_lev['stcounty'] = (high_lev['state']+high_lev['county'])

/var/folders/n2/21777nbs0wx_k3zppskxw6540000gn/T/ipykernel_69851/3344273758.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  high_lev['state'] = high_lev.fipstate.str.rjust(2, '0')
/var/folders/n2/21777nbs0wx_k3zppskxw6540000gn/T/ipykernel_69851/3344273758.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  high_lev['county'] = high_lev.fipscty.str.rjust(3,'0')
/var/folders/n2/21777nbs0wx_k3zppskxw6540000gn/T/ipykernel_69851/3344273758.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy 

In [112]:
high_lev.head()

,fipstate,fipscty,naics,emp_nf,emp,qp1_nf,qp1,ap_nf,ap,est,...,n500_999,n1000,n1000_1,n1000_2,n1000_3,n1000_4,naics_num,state,county,stcounty
0,1,1,------,G,11265,G,94865,G,385785,879,...,N,N,N,N,N,N,0,01,001,01001
1,1,1,11----,H,92,G,1183,H,5232,10,...,N,N,N,N,N,N,11,01,001,01001
7,1,1,21----,G,85,G,1208,G,4874,5,...,N,N,N,N,N,N,21,01,001,01001
12,1,1,22----,H,165,H,5460,H,19533,8,...,N,N,N,N,N,N,22,01,001,01001
17,1,1,23----,G,559,G,6211,G,25764,79,...,N,N,N,N,N,N,23,01,001,01001


In [122]:
## Final DataFrame with only necesarry columns
joined = rep_with_county.merge(high_lev,left_on='GEOID',right_on='stcounty')[['name','role','party'
                                                                              ,'Name','geometry','naics_num'
                                                                              ,'emp','qp1','ap','est']]

In [124]:
joined.naics_num.unique()

array([ 0, 11, 21, 22, 23, 31, 42, 44, 48, 51, 52, 53, 54, 55, 56, 61, 62,
       71, 72, 81, 99])

In [140]:
mapping = {11:'Agriculture, Forestry, Hunting and Fishing'
          ,21:'Mining'
          ,22:'Utilities'
          ,23:'Construction'
          ,31:'Construction'
          ,32:'Construction'
          ,33:'Construction'
          ,42:'Wholesale Trade'
          ,44:'Retail Trade'
          ,45:'Retail Trade'
          ,48:'Transportation and Warehousing'
          ,49:'Transportation and Warehousing'
          ,51:'Information (Media)'
          ,52:'Finance and Insurance'
          ,53:'Real Estate Rental and Leasing'
          ,54:'Professional, Scientific, and Technical Services'
          ,55:'Management of Companies and Enterprises'
          ,56:'Administrative Support and Waste Management'
          ,61:'Education Services'
          ,62:'Health Care and Social Assistance'
          ,71:'Arts, Entertainment, and Recreation'
          ,72:'Accomodation and Food Services'
          ,81:'Other Services (excl. Public Service)'
          ,99:'Other'}
map_df = pd.DataFrame.from_dict(mapping, orient='index',columns=['title']).reset_index()

In [149]:
final_join = joined.merge(map_df,left_on='naics_num',right_on='index')
final_join

,name,role,party,Name,geometry,naics_num,emp,qp1,ap,est,index,title
0,Adline Clarke,Representative,Democratic,Alabama,"POLYGON ((-88.11462 30.68308, -88.11325 30.683...",11,299,2695,12472,40,11,"Agriculture, Forestry, Hunting and Fishing"
1,Allen Treadaway,Representative,Republican,Alabama,"POLYGON ((-86.96357 33.72058, -86.96326 33.721...",11,56,805,3245,10,11,"Agriculture, Forestry, Hunting and Fishing"
2,Anthony Daniels,Representative,Democratic,Alabama,"POLYGON ((-86.67282 34.76187, -86.67280 34.763...",11,201,2341,9544,18,11,"Agriculture, Forestry, Hunting and Fishing"
3,Arnold Mooney,Representative,Republican,Alabama,"POLYGON ((-86.86034 33.31579, -86.86031 33.315...",11,89,690,3064,12,11,"Agriculture, Forestry, Hunting and Fishing"
4,Artis 'A.J.' McCampbell,Representative,Democratic,Alabama,"POLYGON ((-88.42133 32.30975, -88.42095 32.312...",11,12,77,305,5,11,"Agriculture, Forestry, Hunting and Fishing"
...,...,...,...,...,...,...,...,...,...,...,...,...
56783,Landon Brown,Representative,Republican,Wyoming,"POLYGON ((-104.79128 41.17053, -104.79125 41.1...",99,3,11,124,6,99,Other
56784,Lane Allred,Representative,Republican,Wyoming,"POLYGON ((-111.04723 42.46910, -111.04713 42.4...",99,41,142,942,15,99,Other
56785,Liz Storer,Representative,Democratic,Wyoming,"POLYGON ((-111.05345 44.66627, -111.02661 44.6...",99,11,35,316,8,99,Other
56786,Sarah Penn,Representative,Republican,Wyoming,"POLYGON ((-109.75462 43.42970, -109.75462 43.4...",99,14,174,1771,18,99,Other


In [166]:
rep_data = final_join.groupby(['name','role','party','Name','title'],as_index=False).agg({'emp':'sum'
                                                                                          ,'qp1':'sum','ap':'sum'})
rep_data

,name,role,party,Name,title,emp,qp1,ap
0,A. Reece Pyrtle Jr.,Representative,Republican,North Carolina,Accomodation and Food Services,1037,3524,13889
1,A. Reece Pyrtle Jr.,Representative,Republican,North Carolina,Administrative Support and Waste Management,301,1996,10997
2,A. Reece Pyrtle Jr.,Representative,Republican,North Carolina,"Arts, Entertainment, and Recreation",30,86,355
3,A. Reece Pyrtle Jr.,Representative,Republican,North Carolina,Construction,2779,34817,134109
4,A. Reece Pyrtle Jr.,Representative,Republican,North Carolina,Education Services,194,1314,5100
...,...,...,...,...,...,...,...,...
28822,Zack Hawkins,Representative,Democratic,North Carolina,Real Estate Rental and Leasing,514,4164,21673
28823,Zack Hawkins,Representative,Democratic,North Carolina,Retail Trade,4431,29345,126025
28824,Zack Hawkins,Representative,Democratic,North Carolina,Transportation and Warehousing,390,2588,12177
28825,Zack Hawkins,Representative,Democratic,North Carolina,Utilities,130,2485,9033


In [167]:
shapefile = rep_data.merge(district_full, left_on='name',right_on='name')[['name','role_x','party_x','Name_x','title'
                                                              ,'emp','qp1','ap','geometry']]

In [171]:
shapefile.rename(columns={'name':'Rep Name','role_x':'Role','party_x':'Political Party','Name_x':'State'
                 ,'title':'NAICS Sector','emp':'Employment','qp1':'Footprint','ap':'Payroll'},inplace=True)

In [172]:
shapefile

,Rep Name,Role,Political Party,State,NAICS Sector,Employment,Footprint,Payroll,geometry
0,A. Reece Pyrtle Jr.,Representative,Republican,North Carolina,Accomodation and Food Services,1037,3524,13889,"POLYGON ((-80.03512 36.25738, -80.03511 36.257..."
1,A. Reece Pyrtle Jr.,Representative,Republican,North Carolina,Administrative Support and Waste Management,301,1996,10997,"POLYGON ((-80.03512 36.25738, -80.03511 36.257..."
2,A. Reece Pyrtle Jr.,Representative,Republican,North Carolina,"Arts, Entertainment, and Recreation",30,86,355,"POLYGON ((-80.03512 36.25738, -80.03511 36.257..."
3,A. Reece Pyrtle Jr.,Representative,Republican,North Carolina,Construction,2779,34817,134109,"POLYGON ((-80.03512 36.25738, -80.03511 36.257..."
4,A. Reece Pyrtle Jr.,Representative,Republican,North Carolina,Education Services,194,1314,5100,"POLYGON ((-80.03512 36.25738, -80.03511 36.257..."
...,...,...,...,...,...,...,...,...,...
28888,Zack Hawkins,Representative,Democratic,North Carolina,Real Estate Rental and Leasing,514,4164,21673,"POLYGON ((-78.91923 35.96542, -78.91878 35.965..."
28889,Zack Hawkins,Representative,Democratic,North Carolina,Retail Trade,4431,29345,126025,"POLYGON ((-78.91923 35.96542, -78.91878 35.965..."
28890,Zack Hawkins,Representative,Democratic,North Carolina,Transportation and Warehousing,390,2588,12177,"POLYGON ((-78.91923 35.96542, -78.91878 35.965..."
28891,Zack Hawkins,Representative,Democratic,North Carolina,Utilities,130,2485,9033,"POLYGON ((-78.91923 35.96542, -78.91878 35.965..."


In [173]:
shapefile.to_csv('Shapefile_Rep_Sector_data.csv')